In [2]:
pwd

'/Users/netcan/Workspace/Programming/FCEmulator/src/tools'

In [2]:
import InstructionReference

In [3]:
ret = InstructionReference.get_inst_ref()

In [4]:
import pandas as pd
df = pd.DataFrame(ret)

In [5]:
df['addressingMode'] = df['addressingMode'].str.replace('Implied', 'Implicit'
                                ).str.replace('Zero\s+Page', 'ZeroPage'
                                ).str.replace('ZeroPage,X', 'ZeroPageX'
                                ).str.replace('ZeroPage,Y', 'ZeroPageY'
                                ).str.replace('Absolute,X', 'AbsoluteX'
                                ).str.replace('Absolute,Y', 'AbsoluteY'
                                ).str.replace('\(Indirect,X\)', 'IndexIndirect'
                                ).str.replace('\(Indirect\),Y', 'IndirectIndex')
df.head()

,name,addressingMode,code,bytes,cycles,extraCycles
0,ADC,Immediate,$69,2,2,0
1,ADC,ZeroPage,$65,2,3,0
2,ADC,ZeroPageX,$75,2,4,0
3,ADC,Absolute,$6D,3,4,0
4,ADC,AbsoluteX,$7D,3,4 (+1 if page crossed),0


In [6]:
df['code'] = df['code'].str.replace('$', '0x')
df.head()

,name,addressingMode,code,bytes,cycles,extraCycles
0,ADC,Immediate,0x69,2,2,0
1,ADC,ZeroPage,0x65,2,3,0
2,ADC,ZeroPageX,0x75,2,4,0
3,ADC,Absolute,0x6D,3,4,0
4,ADC,AbsoluteX,0x7D,3,4 (+1 if page crossed),0


In [7]:
df.loc[df['cycles'].str.contains('\+1'), 'extraCycles'] = 1
df.loc[df['cycles'].str.contains('\+2'), 'extraCycles'] = 2
df.head()

,name,addressingMode,code,bytes,cycles,extraCycles
0,ADC,Immediate,0x69,2,2,0
1,ADC,ZeroPage,0x65,2,3,0
2,ADC,ZeroPageX,0x75,2,4,0
3,ADC,Absolute,0x6D,3,4,0
4,ADC,AbsoluteX,0x7D,3,4 (+1 if page crossed),1


In [9]:
df['cycles'] = df['cycles'].str.replace('[\n\s]', ''
                            ).str.replace('(\d+).*', lambda s: s.group(1))
df.head()

,name,addressingMode,code,bytes,cycles,extraCycles
0,ADC,Immediate,0x69,2,2,0
1,ADC,ZeroPage,0x65,2,3,0
2,ADC,ZeroPageX,0x75,2,4,0
3,ADC,Absolute,0x6D,3,4,0
4,ADC,AbsoluteX,0x7D,3,4,1


In [25]:
df['name'] = df['name'].str.replace('^\w+$', lambda s: 'OpName::' + s.group(0))
df.head()

,name,addressingMode,code,bytes,cycles,extraCycles
0,OpName::ADC,Immediate,0x69,2,2,0
1,OpName::ADC,ZeroPage,0x65,2,3,0
2,OpName::ADC,ZeroPageX,0x75,2,4,0
3,OpName::ADC,Absolute,0x6D,3,4,0
4,OpName::ADC,AbsoluteX,0x7D,3,4,1


In [26]:
df['addressingMode'] = df['addressingMode'].str.replace('^\w+$', lambda s: 'OpAddressingMode::' + s.group(0))
df.head()

,name,addressingMode,code,bytes,cycles,extraCycles
0,OpName::ADC,OpAddressingMode::Immediate,0x69,2,2,0
1,OpName::ADC,OpAddressingMode::ZeroPage,0x65,2,3,0
2,OpName::ADC,OpAddressingMode::ZeroPageX,0x75,2,4,0
3,OpName::ADC,OpAddressingMode::Absolute,0x6D,3,4,0
4,OpName::ADC,OpAddressingMode::AbsoluteX,0x7D,3,4,1


In [61]:
for (_, row) in df.iterrows():
    print('\t{{{}, {}, {}, {}, {}, {}, nullptr}}, '
        .format(row['name'], row['addressingMode'],
                row['code'], row['bytes'],
                row['cycles'], row['extraCycles']))

	{OpName::ADC, OpAddressingMode::Immediate, 0x69, 2, 2, 0, nullptr}, 
	{OpName::ADC, OpAddressingMode::ZeroPage, 0x65, 2, 3, 0, nullptr}, 
	{OpName::ADC, OpAddressingMode::ZeroPageX, 0x75, 2, 4, 0, nullptr}, 
	{OpName::ADC, OpAddressingMode::Absolute, 0x6D, 3, 4, 0, nullptr}, 
	{OpName::ADC, OpAddressingMode::AbsoluteX, 0x7D, 3, 4, 1, nullptr}, 
	{OpName::ADC, OpAddressingMode::AbsoluteY, 0x79, 3, 4, 1, nullptr}, 
	{OpName::ADC, OpAddressingMode::IndexIndirect, 0x61, 2, 6, 0, nullptr}, 
	{OpName::ADC, OpAddressingMode::IndirectIndex, 0x71, 2, 5, 1, nullptr}, 
	{OpName::AND, OpAddressingMode::Immediate, 0x29, 2, 2, 0, nullptr}, 
	{OpName::AND, OpAddressingMode::ZeroPage, 0x25, 2, 3, 0, nullptr}, 
	{OpName::AND, OpAddressingMode::ZeroPageX, 0x35, 2, 4, 0, nullptr}, 
	{OpName::AND, OpAddressingMode::Absolute, 0x2D, 3, 4, 0, nullptr}, 
	{OpName::AND, OpAddressingMode::AbsoluteX, 0x3D, 3, 4, 1, nullptr}, 
	{OpName::AND, OpAddressingMode::AbsoluteY, 0x39, 3, 4, 1, nullptr}, 
	{OpName::AND, O